# Notebook de Scrapping de données 
## Le scrapping
On va utiliser la librairy BeautifulSoup pour scrapper (récupérer) les données à partir d'un site web, puis les insérer dans un fichier CSV afin de les exploités.
## Les données
On récupère les données depuis le site web d'allociné. Il contient la plus grosse base francophone de films. On va y récupéré une très grosse quantité de données. 

In [7]:
# Importation des bibliothèques
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
import base64

In [68]:
# On récupère l'url de la page
url = "https://www.allocine.fr/films/decennie-2000/?page="

In [9]:
# On test si l'url est valide avec une requête HTTP afin de récupérer le code source de la page
response = requests.get(url)
response.raise_for_status()

In [10]:
# Si la requête est valide, on récupère le code source de la page
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8">
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="noindex, nofollow" name="robots">
<title>Les films les plus populaires - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Découvrez tous les films les plus populaires sur AlloCiné, comme :  Un homme est mort, Josey Wales hors la loi, Emmanuelle" name="description"/>
<meta content="AlloCiné" property="og:site_name"/>
<meta content="fr_FR" property="og:

In [11]:
# Fonction pour scraper une seule page
def scrape_page(page):
    response = requests.get(url + str(page))
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'lxml')
    page_movies = []

    for movie in soup.select('li.mdl'):
        title = movie.select_one('h2.meta-title').text.strip() if movie.select_one('h2.meta-title') else ''
        # On récupère l'url de l'affiche du film qui peut être soit en 'src' soit en 'data-src'
        img_tag = movie.select_one('img.thumbnail-img')
        if img_tag:
            poster = img_tag.get('data-src', img_tag.get('src', ''))
            if poster.startswith('data:image'):
                # Enregistrer l'image encodée en base64
                filename = f"poster_{title.replace(' ', '_')}.jpg"
                save_base64_image(poster, filename)
                poster = filename
        else:
            poster = ''
        rating = movie.select_one('span.stareval-note').text.strip() if movie.select_one('span.stareval-note') else ''
        author = movie.select_one('div.meta-body-item.meta-body-direction').text.strip().replace('\n', ' ').replace('|', ' ').replace('  ', ' ').replace('De', '') if movie.select_one('div.meta-body-item.meta-body-direction') else ''
        infos = movie.select_one('div.meta-body-item.meta-body-info').text.strip().split('\n|\n') if movie.select_one('div.meta-body-item.meta-body-info') else []
        date = infos[0] if len(infos) > 0 else ''
        duration = infos[1] if len(infos) > 1 else ''
        genre = infos[2].replace('\n', '') if len(infos) > 2 else ''
        actors = movie.select_one('div.meta-body-item.meta-body-actor').text.strip().replace('\n', ' ').replace('|', ' ').replace('  ', ' ').replace('Avec', '').replace(', ', ',') if movie.select_one('div.meta-body-item.meta-body-actor') else ''
        synopsis = movie.select_one('div.synopsis').text.strip() if movie.select_one('div.synopsis') else ''

        page_movies.append({
            'title': title,
            'poster': poster,
            'author': author,
            'date': date,
            'rating': rating,
            'genre': genre,
            'actors': actors,
            'duration': duration,
            'synopsis': synopsis
        })
    
    return page_movies

In [12]:
# Scraper plusieurs pages en parallèle
def scrape_all_pages(start_page, end_page):
    movies = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(scrape_page, page) for page in range(start_page, end_page + 1)]
        for future in as_completed(futures):
            movies.extend(future.result())
    return movies

In [117]:
# On boucle sur le scraping de toutes les pages
url = "https://www.allocine.fr/films/decennie-2000/?page="
start = 0
end = 1531
for i in range(start, end, 100):
    print(f'Scraping des pages {i} à {i + 99}...')
    movies = scrape_all_pages(i, i + 99)
    print(f'Scraping pages Réussie !')
    # On écrit les données dans un fichier CSV
    with open('movies.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=movies[0].keys())
        if i == start:
            writer.writeheader()
        writer.writerows(movies)

Scraping des pages 0 à 99...
Scraping pages Réussie !
Scraping des pages 100 à 199...
Scraping pages Réussie !
Scraping des pages 200 à 299...
Scraping pages Réussie !
Scraping des pages 300 à 399...
Scraping pages Réussie !
Scraping des pages 400 à 499...
Scraping pages Réussie !
Scraping des pages 500 à 599...
Scraping pages Réussie !
Scraping des pages 600 à 699...
Scraping pages Réussie !
Scraping des pages 700 à 799...
Scraping pages Réussie !
Scraping des pages 800 à 899...
Scraping pages Réussie !
Scraping des pages 900 à 999...
Scraping pages Réussie !
Scraping des pages 1000 à 1099...
Scraping pages Réussie !
Scraping des pages 1100 à 1199...
Scraping pages Réussie !
Scraping des pages 1200 à 1299...
Scraping pages Réussie !
Scraping des pages 1300 à 1399...
Scraping pages Réussie !
Scraping des pages 1400 à 1499...
Scraping pages Réussie !
Scraping des pages 1500 à 1599...
Scraping pages Réussie !


In [101]:
# On boucle sur le scraping de toutes les pages 2010
url = "https://www.allocine.fr/films/decennie-2010/?page="
start = 0
end = 1768
for i in range(start, end, 100):
    print(f'Scraping des pages {i} à {i + 99}...')
    movies = scrape_all_pages(i, i + 99)
    print(f'Scraping pages Réussie !')
    # On écrit les données dans un fichier CSV
    with open('movies.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=movies[0].keys())
        if i == start:
            writer.writeheader()
        writer.writerows(movies)

Scraping des pages 0 à 99...
Scraping pages Réussie !
Scraping des pages 100 à 199...
Scraping pages Réussie !
Scraping des pages 200 à 299...
Scraping pages Réussie !
Scraping des pages 300 à 399...
Scraping pages Réussie !
Scraping des pages 400 à 499...
Scraping pages Réussie !
Scraping des pages 500 à 599...
Scraping pages Réussie !
Scraping des pages 600 à 699...
Scraping pages Réussie !
Scraping des pages 700 à 799...
Scraping pages Réussie !
Scraping des pages 800 à 899...
Scraping pages Réussie !
Scraping des pages 900 à 999...
Scraping pages Réussie !
Scraping des pages 1000 à 1099...
Scraping pages Réussie !
Scraping des pages 1100 à 1199...
Scraping pages Réussie !
Scraping des pages 1200 à 1299...
Scraping pages Réussie !
Scraping des pages 1300 à 1399...
Scraping pages Réussie !
Scraping des pages 1400 à 1499...
Scraping pages Réussie !
Scraping des pages 1500 à 1599...
Scraping pages Réussie !
Scraping des pages 1600 à 1699...
Scraping pages Réussie !
Scraping des pages 1

In [135]:
# On boucle sur le scraping de toutes les pages 2020
url = "https://www.allocine.fr/films/decennie-2020/?page="
start = 0
end = 1116
for i in range(start, end, 100):
    print(f'Scraping des pages {i} à {i + 99}...')
    movies = scrape_all_pages(i, i + 99)
    print(f'Scraping pages Réussie !')
    # On écrit les données dans un fichier CSV
    with open('movies.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=movies[0].keys())
        if i == start:
            writer.writeheader()
        writer.writerows(movies)

Scraping des pages 0 à 99...
Scraping pages Réussie !
Scraping des pages 100 à 199...
Scraping pages Réussie !
Scraping des pages 200 à 299...
Scraping pages Réussie !
Scraping des pages 300 à 399...
Scraping pages Réussie !
Scraping des pages 400 à 499...
Scraping pages Réussie !
Scraping des pages 500 à 599...
Scraping pages Réussie !
Scraping des pages 600 à 699...
Scraping pages Réussie !
Scraping des pages 700 à 799...
Scraping pages Réussie !
Scraping des pages 800 à 899...
Scraping pages Réussie !
Scraping des pages 900 à 999...
Scraping pages Réussie !
Scraping des pages 1000 à 1099...
Scraping pages Réussie !
Scraping des pages 1100 à 1199...
Scraping pages Réussie !


In [136]:
# On test si les informations sont correctes en affichant les 5 premiers films
for movie in movies[:5]:
    print(movie)


{'title': 'Almost Had You', 'poster': 'https://fr.web.img4.acsta.net/c_310_420/pictures/22/06/10/17/51/0792577.jpg', 'author': ' Richard Omos-Iboyi', 'date': 'Date de sortie inconnue', 'rating': '', 'genre': '', 'actors': '', 'duration': 'Drame,\nRomance', 'synopsis': ''}
{'title': 'A Far Shore', 'poster': 'https://fr.web.img6.acsta.net/c_310_420/commons/v9/common/empty/empty_portrait.png', 'author': ' Masaaki Kudô', 'date': 'Prochainement', 'rating': '', 'genre': 'Drame', 'actors': ' Kotone Hanase,Yumemi Ishida,Yoshiro Sakuma', 'duration': '2h 08min', 'synopsis': 'Aoi, 17 ans, travaille comme hôtesse dans une boîte de nuit afin de subvenir aux besoins de son petit garçon mais aussi de Masaya, son mari paresseux et violent. La jeune femme n’a guère d’horizon, perdue à Okinawa. Quelle chance Aoi a-t-elle de se sortir de ce marasme social ?'}
{'title': 'La Balada del fénix', 'poster': 'https://fr.web.img6.acsta.net/c_310_420/commons/v9/common/empty/empty_portrait.png', 'author': ' Arturo

In [142]:
# On importe numpy et pandas pour afficher les informations du fichier CSV
import numpy as np
import pandas as pd

# On lit le fichier CSV
df = pd.read_csv('movies.csv', delimiter=',')
df

,title,poster,author,date,rating,genre,actors,duration,synopsis
0,Viêt and Nam,https://fr.web.img2.acsta.net/c_310_420/img/e1...,Minh Quý Trương,25 septembre 2024,"3,4",Drame,"Thanh Hai Pham,Duy Bao Dinh Dao,Viet Tung Le",2h 09min,Nam et Viêt s’aiment. Tous les deux travaillen...
1,"Moi, Moche et Méchant 4",https://fr.web.img5.acsta.net/c_310_420/img/cb...,"Patrick lage, Chris Renaud",10 juillet 2024,"2,7","Aventure,Animation,Comédie","Gad Elmaleh,Steve Carell,Audrey Lamy",1h 34min,"Gru, Lucy et les filles, Margo, Edith et Agnès..."
2,The Deliverance,https://fr.web.img6.acsta.net/c_310_420/img/ee...,Lee Daniels,30 août 2024,"2,3","Epouvante-horreur,Thriller","Andra Day,Glenn Close,Anthony B. Jenkins",1h 51min,Ebony et ses trois enfants rencontrent des inc...
3,Bullet Train,https://fr.web.img6.acsta.net/c_310_420/pictur...,David Leitch,3 août 2022,"3,2","Action,Thriller","Brad Pitt,Joey King,Aaron Taylor-Johnson",2h 07min,Coccinelle est un assassin malchanceux et part...
4,Pauvres Créatures,https://fr.web.img5.acsta.net/c_310_420/pictur...,Yorgos Lanthimos,17 janvier 2024,"3,7","Comédie,Drame,Fantastique","Emma Stone,Mark Ruffalo,Willem Dafoe",2h 21min,Bella est une jeune femme ramenée à la vie par...
...,...,...,...,...,...,...,...,...,...
17991,Emmanuelle,https://fr.web.img2.acsta.net/c_310_420/img/b6...,Audrey Diwan,25 septembre 2024,"3,0","Drame,Erotique","Noémie Merlant,Will Sharpe,Naomi Watts",1h 47min,Emmanuelle est en quête d’un plaisir perdu. El...
17992,Les Barbares,https://fr.web.img4.acsta.net/c_310_420/img/08...,Julie lpy,18 septembre 2024,"3,3",Comédie,"Julie Delpy,Sandrine Kiberlain,Laurent Lafitte",1h 41min,"A Paimpont, l’harmonie règne : parmi les habit..."
17993,Tatami,https://fr.web.img6.acsta.net/c_310_420/img/46...,"Zar Amir Ebrahimi, Guy Nattiv",4 septembre 2024,"3,8",Drame,"Arienne Mandi,Zar Amir Ebrahimi,Ash Goldeh",1h 43min,La judokate iranienne Leila et son entraîneuse...
17994,Speak No Evil,https://fr.web.img5.acsta.net/c_310_420/img/3b...,James Watkins,18 septembre 2024,"3,5",Epouvante-horreur,"James McAvoy,Mackenzie Davis,Aisling Franciosi",1h 50min,Une famille américaine passe le week-end dans ...


In [143]:
# On supprimme les films qui n'ont pas de genre ou de realisateur
df = df.dropna(subset=['genre', 'author'])
# On supprimme les doublons
df = df.drop_duplicates(subset=['title'])

In [144]:
# On supprime les films avec date : date de sortie inconnue
df = df[df['date'] != 'Date de sortie inconnue']
df = df[df['date'] != 'date']
df = df[df['date'] != 'Prochainement']
df

,title,poster,author,date,rating,genre,actors,duration,synopsis
0,Viêt and Nam,https://fr.web.img2.acsta.net/c_310_420/img/e1...,Minh Quý Trương,25 septembre 2024,"3,4",Drame,"Thanh Hai Pham,Duy Bao Dinh Dao,Viet Tung Le",2h 09min,Nam et Viêt s’aiment. Tous les deux travaillen...
1,"Moi, Moche et Méchant 4",https://fr.web.img5.acsta.net/c_310_420/img/cb...,"Patrick lage, Chris Renaud",10 juillet 2024,"2,7","Aventure,Animation,Comédie","Gad Elmaleh,Steve Carell,Audrey Lamy",1h 34min,"Gru, Lucy et les filles, Margo, Edith et Agnès..."
2,The Deliverance,https://fr.web.img6.acsta.net/c_310_420/img/ee...,Lee Daniels,30 août 2024,"2,3","Epouvante-horreur,Thriller","Andra Day,Glenn Close,Anthony B. Jenkins",1h 51min,Ebony et ses trois enfants rencontrent des inc...
3,Bullet Train,https://fr.web.img6.acsta.net/c_310_420/pictur...,David Leitch,3 août 2022,"3,2","Action,Thriller","Brad Pitt,Joey King,Aaron Taylor-Johnson",2h 07min,Coccinelle est un assassin malchanceux et part...
4,Pauvres Créatures,https://fr.web.img5.acsta.net/c_310_420/pictur...,Yorgos Lanthimos,17 janvier 2024,"3,7","Comédie,Drame,Fantastique","Emma Stone,Mark Ruffalo,Willem Dafoe",2h 21min,Bella est une jeune femme ramenée à la vie par...
...,...,...,...,...,...,...,...,...,...
14749,Les biscuits magiques de Noël,https://fr.web.img6.acsta.net/c_310_420/img/54...,Michael M. Scott,23 octobre 2024,NaN,Romance,"Merritt Patterson,Jon-Michael Ecker,Jorge Mon...",1h 23min,NaN
15133,Kala azar,https://fr.web.img6.acsta.net/c_310_420/common...,Janis Rafailidou,2 juin 2021,"2,5",Drame,"Penelope Tsilika,Dimitris Lalos,Michele Valley",1h 32min,Penelope et Dimitris sont un couple de crémate...
15764,Où est passé Noël ?,https://fr.web.img6.acsta.net/c_310_420/img/57...,Emily Moss Wilson,21 octobre 2024,NaN,Romance,"Rachael Leigh Cook,Sam Page,Bailey Stender",1h 24min,NaN
16464,Un Noël presque royal !,https://fr.web.img3.acsta.net/c_310_420/img/ac...,Jonathan Wright,22 octobre 2024,NaN,Romance,"Brooke d'Orsay,Will Kemp,Taylor Love",1h 24min,NaN


In [145]:
# On efface les films dont les 2 derniers chiffres de la date de sortie ne se situent pas entre 70 et 79
df = df[df['date'].str[-2:].astype(int).between(20, 24)]
df

,title,poster,author,date,rating,genre,actors,duration,synopsis
0,Viêt and Nam,https://fr.web.img2.acsta.net/c_310_420/img/e1...,Minh Quý Trương,25 septembre 2024,"3,4",Drame,"Thanh Hai Pham,Duy Bao Dinh Dao,Viet Tung Le",2h 09min,Nam et Viêt s’aiment. Tous les deux travaillen...
1,"Moi, Moche et Méchant 4",https://fr.web.img5.acsta.net/c_310_420/img/cb...,"Patrick lage, Chris Renaud",10 juillet 2024,"2,7","Aventure,Animation,Comédie","Gad Elmaleh,Steve Carell,Audrey Lamy",1h 34min,"Gru, Lucy et les filles, Margo, Edith et Agnès..."
2,The Deliverance,https://fr.web.img6.acsta.net/c_310_420/img/ee...,Lee Daniels,30 août 2024,"2,3","Epouvante-horreur,Thriller","Andra Day,Glenn Close,Anthony B. Jenkins",1h 51min,Ebony et ses trois enfants rencontrent des inc...
3,Bullet Train,https://fr.web.img6.acsta.net/c_310_420/pictur...,David Leitch,3 août 2022,"3,2","Action,Thriller","Brad Pitt,Joey King,Aaron Taylor-Johnson",2h 07min,Coccinelle est un assassin malchanceux et part...
4,Pauvres Créatures,https://fr.web.img5.acsta.net/c_310_420/pictur...,Yorgos Lanthimos,17 janvier 2024,"3,7","Comédie,Drame,Fantastique","Emma Stone,Mark Ruffalo,Willem Dafoe",2h 21min,Bella est une jeune femme ramenée à la vie par...
...,...,...,...,...,...,...,...,...,...
14749,Les biscuits magiques de Noël,https://fr.web.img6.acsta.net/c_310_420/img/54...,Michael M. Scott,23 octobre 2024,NaN,Romance,"Merritt Patterson,Jon-Michael Ecker,Jorge Mon...",1h 23min,NaN
15133,Kala azar,https://fr.web.img6.acsta.net/c_310_420/common...,Janis Rafailidou,2 juin 2021,"2,5",Drame,"Penelope Tsilika,Dimitris Lalos,Michele Valley",1h 32min,Penelope et Dimitris sont un couple de crémate...
15764,Où est passé Noël ?,https://fr.web.img6.acsta.net/c_310_420/img/57...,Emily Moss Wilson,21 octobre 2024,NaN,Romance,"Rachael Leigh Cook,Sam Page,Bailey Stender",1h 24min,NaN
16464,Un Noël presque royal !,https://fr.web.img3.acsta.net/c_310_420/img/ac...,Jonathan Wright,22 octobre 2024,NaN,Romance,"Brooke d'Orsay,Will Kemp,Taylor Love",1h 24min,NaN


In [146]:
df.to_csv('movies2020.csv', index=False)

In [148]:
# On récupère tous les csv (70 80 90 2000 2010 2020) et on les fusionne
df70 = pd.read_csv('movies70.csv', delimiter=',')
df80 = pd.read_csv('movies80.csv', delimiter=',')
df90 = pd.read_csv('movies90.csv', delimiter=',')
df2000 = pd.read_csv('movies2000.csv', delimiter=',')
df2010 = pd.read_csv('movies2010.csv', delimiter=',')
df2020 = pd.read_csv('movies2020.csv', delimiter=',')
df_cleaned = pd.concat([df70, df80, df90, df2000, df2010, df2020])
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24874 entries, 0 to 6258
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     24874 non-null  object
 1   poster    24874 non-null  object
 2   author    24874 non-null  object
 3   date      24874 non-null  object
 4   rating    23894 non-null  object
 5   genre     24874 non-null  object
 6   actors    24540 non-null  object
 7   duration  24874 non-null  object
 8   synopsis  23885 non-null  object
dtypes: object(9)
memory usage: 1.9+ MB


In [149]:
# on cope le fichier dataframe dans un fichier CSV
df_cleaned.to_csv('movies_cleaned.csv', index=False)